# Parity checks on DMs

Creating calibration data for flips and rotations between the two DMs and the final focal plane.

In [ ]:
# Imports
%matplotlib inline
import os
from astropy.io import fits
import matplotlib as mpl
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np

from Asterix import Asterix_root
from Asterix.utils import read_parameter_file
from Asterix.optics import Pupil, Coronagraph, DeformableMirror, Testbed
from Asterix.main_THD import THD2

In [ ]:
# Some setup for pretty plotting
mpl.rc('image', origin='lower',   # Put the origin in the lower left corner.
       interpolation=None)        # Do not interpolate between pixels in the display.

In [ ]:
# Load the template parameter file
parameter_file_ex = os.path.join(Asterix_root, "Example_param_file.ini")
config = read_parameter_file(parameter_file_ex)

In [ ]:
thd2 = THD2(parameter_file_ex)

## Focal-plane images with applied DM commands

We first create global tip and tilt commands for both DMs. We then apply these individually to the DMs to see in which direction the non-coronagraphic PSF moves. You always need to note the image origin in such tests, here I set the image origin permanently to the lower left at the beginning of this notebook.

In [ ]:
# Ramp on all actuators of either DM, creating a tip/tilt
shift_x = 1000   # no clue what units these create here, but this is a decent aplitude for the iamges below
shift_y = 1000

maskx_dm1 = np.linspace(-np.pi * shift_x, np.pi * shift_x, 34, endpoint=False)
masky_dm1 = np.linspace(-np.pi * shift_y, np.pi * shift_y, 34, endpoint=False)
xx_dm1, yy_dm1 = np.meshgrid(maskx_dm1, masky_dm1)
ramp_dm1 = xx_dm1
ramp_dm1_rot = yy_dm1

maskx_dm3 = np.linspace(-np.pi * shift_x, np.pi * shift_x, 32, endpoint=False)
masky_dm3 = np.linspace(-np.pi * shift_y, np.pi * shift_y, 32, endpoint=False)
xx_dm3, yy_dm3 = np.meshgrid(maskx_dm3, masky_dm3)
ramp_dm3 = xx_dm3
ramp_dm3_rot = yy_dm3

print(type(ramp_dm1))
print(ramp_dm1.shape)
print(type(ramp_dm3))
print(ramp_dm3.shape)

plt.figure(figsize=(12, 12))
plt.subplot(2, 2, 1)
plt.imshow(ramp_dm1, cmap='RdBu')
plt.title('ramp_dm1 command DM1')
plt.colorbar()

plt.subplot(2, 2, 2)
plt.imshow(ramp_dm3, cmap='RdBu')
plt.title('ramp_dm3 command DM3')
plt.colorbar()

plt.subplot(2, 2, 3)
plt.imshow(ramp_dm1_rot, cmap='RdBu')
plt.title('ramp_dm1_rot command DM1')
plt.colorbar()

plt.subplot(2, 2, 4)
plt.imshow(ramp_dm3_rot, cmap='RdBu')
plt.title('ramp_dm3_rot command DM3')
plt.colorbar()

In [ ]:
print(thd2.number_act)

In [ ]:
num_act_dm1 = 952
num_act_dm3 = 1024  # 713?

The total DM command is a concatenated array of the separate DM1 and DM3 commands. Since DM1 is not a square array, we need to correctly pick the part of the ramps above that are the actual DM1 actuators on the grid. We can use this by using a square 2D mask where only the real actuators have a value of one.

I didn't want to add a new fits file to the repo just for this, so I create the mask in situ.

In [ ]:
dm1_mask = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
                     [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
                     [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
                     [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
                     [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
                     [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
                     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                     [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
                     [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
                     [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
                     [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
                     [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
                     [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

plt.imshow(dm1_mask, cmap='Greys_r')
plt.colorbar()

In [ ]:
# If you want a ramp on DM1, change the (un)comment here with ramp_dm1 or ramp_dm1_rot
dm1_command = np.zeros((num_act_dm1))
#dm1_command = ramp_dm1_rot[np.where(dm1_mask)]

# If you want a ramp on DM3, change the (un)comment here with ramp_dm3 or ramp_dm3_rot
dm3_command = np.zeros((num_act_dm3))
#dm3_command = ramp_dm3.ravel()

print(dm1_command.shape)
print(dm3_command.shape)

# Create full DM command
dm_command = np.concatenate((dm1_command, dm3_command))
print(dm_command.shape)

For the below, we use a non-coronagraphic PSF (by setting `noFPM=True`). The un-tilted PSF remains in the image center - it is easier to see the shifts imposed by the phase ramp like this

Note the image origin in the lower left.

In [ ]:
coro_dh = thd2.todetector_intensity(in_contrast=True, voltage_vector=dm_command, noFPM=True)

In [ ]:
plt.figure(figsize=(6, 6))
plt.imshow(coro_dh, cmap='inferno', norm=LogNorm(vmin=1e-6, vmax=1e-2))
plt.colorbar()